In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlrd
import random
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

In [ ]:
data_set = '091902'
# input_file = "combined_20230919061630_Himawari_202309190610.csv"




把科學記號改成十進位

In [ ]:
# import pandas as pd
# import decimal
# 
# # 讀取CSV文件，確保資料的精度
# df = pd.read_csv('data/combined_dataset_all.csv', encoding='utf-8-sig', dtype=str)
# 
# # 將Chlorophyll欄位轉換為不使用科學記號的小數形式
# df['Chlorophyll'] = df['Chlorophyll'].apply(lambda x: format(decimal.Decimal(x), 'f'))
# 
# # 輸出篩選後的結果到原本的CSV文件
# df.to_csv('data/combined_dataset_all.csv', index=False, encoding='utf-8-sig')
# 

先取出1000000筆來train看看

In [ ]:
import pandas as pd

# 讀取CSV文件
combined_df = pd.read_csv(f"data_all/{data_set}.csv")

# 隨機選取10000筆資料
sampled_df = combined_df.sample(n=1000000, random_state=1)

# 保存為新的CSV文件
sampled_df.to_csv("data_all/train_temp/train_combined_dataset_all.csv", index=False)

print("已成功從 combined_dataset_all.csv 中選取 1000000 筆資料並保存到 data/train_combined_dataset_all.csv")


In [ ]:
data = pd.read_csv("data_all/train_temp/train_combined_dataset_all.csv")

In [ ]:
data.info()
data.describe()
data.shape
print(data.columns)

### output filtered csv

In [ ]:
# 選擇所需的列
selected_columns = ['Albedo_01', 'Albedo_02', 'Albedo_03', 'Albedo_04', 'Chlorophyll']

# 過濾數據
filtered_data = data[selected_columns]

## 重命名列，去掉 '2km_'
#filtered_data.columns = ['albedo_01', 'albedo_02', 'albedo_03', 'albedo_04', 'Chlorophyll']
#
# 將過濾後的數據寫入新的CSV文件
filtered_data.to_csv("data_all/train_temp/train_nn_filtered_data.csv", index=False)

print("新的 CSV 文件已成功輸出，文件名為 data_all/train_temp/train_nn_filtered_data.csv")

移除包含 '<' 符號的行(因非海保署資料，故已不再需要)

In [ ]:
# import pandas as pd
# 
# # 讀取CSV文件，確保資料的精度
# df = pd.read_csv('data/train_nn_filtered_data.csv', encoding='utf-8-sig', dtype=str)
# 
# # 移除包含 '<' 符號的行
# filtered_df = df[~df.apply(lambda row: row.str.contains('<|,').any(), axis=1)]
# 
# # 輸出篩選後的結果到新的CSV文件
# filtered_df.to_csv('data/train_filtered_and_trimmed_data.csv', index=False, encoding='utf-8-sig')
# 

科學記號轉小數看看

In [ ]:
import pandas as pd
import decimal

# 讀取CSV文件，確保資料的精度
df = pd.read_csv('data_all/train_temp/train_nn_filtered_data.csv', encoding='utf-8-sig', dtype=str)

# 將Chlorophyll欄位轉換為不使用科學記號的小數形式
df['Chlorophyll'] = df['Chlorophyll'].apply(lambda x: format(decimal.Decimal(x), 'f'))

# 輸出篩選後的結果到原本的CSV文件
df.to_csv('data_all/train_temp/train_nn_filtered_data.csv', index=False, encoding='utf-8-sig')

資料的描述以及葉綠素值的分布圖

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 讀取CSV文件
data = pd.read_csv("data_all/train_temp/train_nn_filtered_data.csv")

# 確認數據的資訊
data.info()

# 顯示基本統計描述
print(data.describe())

# 顯示數據的形狀
print(data.shape)

# 顯示數據的欄位名稱
print(data.columns)

# 畫出Chlorophyll的分布圖
plt.figure(figsize=(10, 6))
plt.hist(data['Chlorophyll'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of Chlorophyll')
plt.xlabel('Chlorophyll')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()







去除極端值

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 讀取CSV文件
data = pd.read_csv("data_all/train_temp/train_nn_filtered_data.csv")

# 顯示原始數據的描述性統計量
print("原始數據的描述性統計量:")
print(data['Chlorophyll'].describe())

# 去除Chlorophyll值大於10的極端值
data_filtered = data[data['Chlorophyll'] <= 30]

# 顯示過濾後數據的描述性統計量
print("\n過濾後數據的描述性統計量:")
print(data_filtered['Chlorophyll'].describe())

# 畫出過濾後Chlorophyll的分布圖
plt.figure(figsize=(10, 2))
plt.hist(data_filtered['Chlorophyll'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of Chlorophyll (Filtered)')
plt.xlabel('Chlorophyll')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# 將過濾後的數據保存到新的CSV文件中
data_filtered.to_csv("data_all/train_temp/train_nn_filtered_data_cleaned.csv", index=False)


count

In [ ]:
print(data_filtered.Chlorophyll.count())
print(data_filtered.Albedo_01.count())
print(data_filtered.Albedo_02.count())
print(data_filtered.Albedo_03.count())
print(data_filtered.Albedo_04.count())

In [ ]:
train = data_filtered
train.info()
train.Chlorophyll.value_counts()

In [ ]:
X = train.drop('Chlorophyll', axis=1)
y = train.Chlorophyll
y.value_counts()

### Scalar

In [ ]:
scaler = StandardScaler()
# scale the data to make it easier for the model to learn
X = scaler.fit_transform(X)

### Train

In [ ]:
test_ratio = 0.2

random_seed = random.randint(1, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_ratio, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(100,)],
    'alpha': [0.0001, 0.001, 0.01, 0.05],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'activation': ['tanh', 'relu']
}



In [ ]:
# # Create an MLPRegressor instance
# mlp = MLPRegressor(solver='sgd', max_iter=200,
#                    n_iter_no_change=10, tol=0.0001, verbose=1)
# 
# # Create a GridSearchCV instance
# grid_mlp = GridSearchCV(
#     mlp, param_grid, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1, cv=5)
# 
# # Fit the grid search to the data
# grid_mlp.fit(X_train, y_train)
# 
# # Get the best estimator
# optimized_mlp = grid_mlp.best_estimator_

NN模型第二版

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(100,), (100, 50), (100, 100, 50)],
    'alpha': [0.0001, 0.001, 0.01, 0.05],
    'learning_rate_init': [0.0001, 0.001, 0.01, 0.05],
    'activation': ['tanh', 'relu']
}
mlp = MLPRegressor(solver='adam', max_iter=500, n_iter_no_change=20, tol=0.0001, verbose=1)
grid_mlp = GridSearchCV(mlp, param_grid, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1, cv=5)
grid_mlp.fit(X_train, y_train)
# Get the best estimator
optimized_mlp = grid_mlp.best_estimator_

In [ ]:
# Print the best parameters found by the grid search
print("MLP best parameters:", grid_mlp.best_params_)

# Make predictions
MLP_train_pred = optimized_mlp.predict(X_train)
MLP_test_pred = optimized_mlp.predict(X_test)


# 將預測結果限制在0到30之間
MLP_train_pred = np.clip(MLP_train_pred, 0, 30)
MLP_test_pred = np.clip(MLP_test_pred, 0, 30)

# Calculate the desired metrics
train_mae = mean_absolute_error(y_train, MLP_train_pred)
test_mae = mean_absolute_error(y_test, MLP_test_pred)
train_rmse = sqrt(mean_squared_error(y_train, MLP_train_pred))
test_rmse = sqrt(mean_squared_error(y_test, MLP_test_pred))
train_r2 = r2_score(y_train, MLP_train_pred)
test_r2 = r2_score(y_test, MLP_test_pred)

# Print the calculated metrics
print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")
print(f"Train RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")
print(f"Train R2: {train_r2}")
print(f"Test R2: {test_r2}")

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already run the previous code and have MLP_train_pred and MLP_test_pred ready

# Combine actual and predicted values for train and test sets
train_results = pd.DataFrame({'Actual': y_train, 'Predicted': MLP_train_pred})
test_results = pd.DataFrame({'Actual': y_test, 'Predicted': MLP_test_pred})

# Plot actual vs predicted for the training set
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.scatter(train_results['Actual'], train_results['Predicted'], alpha=0.5, color='blue')
plt.plot([train_results['Actual'].min(), train_results['Actual'].max()], 
         [train_results['Actual'].min(), train_results['Actual'].max()], 'r--', lw=2)
plt.title('Actual vs Predicted for Training Set')
plt.xlabel('Actual Chlorophyll')
plt.ylabel('Predicted Chlorophyll')
plt.grid(True)

# Plot actual vs predicted for the test set
plt.subplot(1, 2, 2)
plt.scatter(test_results['Actual'], test_results['Predicted'], alpha=0.5, color='green')
plt.plot([test_results['Actual'].min(), test_results['Actual'].max()], 
         [test_results['Actual'].min(), test_results['Actual'].max()], 'r--', lw=2)
plt.title('Actual vs Predicted for Test Set')
plt.xlabel('Actual Chlorophyll')
plt.ylabel('Predicted Chlorophyll')
plt.grid(True)

plt.tight_layout()
plt.show()


輸出不同解析度的圖

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 假設你已經有 y_train, y_test, MLP_train_pred 和 MLP_test_pred 的數據

# Combine actual and predicted values for train and test sets
train_results = pd.DataFrame({'Actual': y_train, 'Predicted': MLP_train_pred})
test_results = pd.DataFrame({'Actual': y_test, 'Predicted': MLP_test_pred})

# alpha值的列表
alpha_values = [0.5, 0.1, 0.04, 0.01, 0.008, 0.004, 0.002]

# 使用迴圈來生成每個alpha值的圖
for alpha in alpha_values:
    # Plot actual vs predicted for the training set
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.scatter(train_results['Actual'], train_results['Predicted'], alpha=alpha, color='blue')
    plt.plot([train_results['Actual'].min(), train_results['Actual'].max()], 
             [train_results['Actual'].min(), train_results['Actual'].max()], 'r--', lw=2)
    plt.title(f'Actual vs Predicted for Training Set (alpha={alpha})')
    plt.xlabel('Actual Chlorophyll')
    plt.ylabel('Predicted Chlorophyll')
    plt.grid(True)

    # Plot actual vs predicted for the test set
    plt.subplot(1, 2, 2)
    plt.scatter(test_results['Actual'], test_results['Predicted'], alpha=alpha, color='green')
    plt.plot([test_results['Actual'].min(), test_results['Actual'].max()], 
             [test_results['Actual'].min(), test_results['Actual'].max()], 'r--', lw=2)
    plt.title(f'Actual vs Predicted for Test Set (alpha={alpha})')
    plt.xlabel('Actual Chlorophyll')
    plt.ylabel('Predicted Chlorophyll')
    plt.grid(True)

    plt.tight_layout()
    plt.show()
    if alpha == 0.002:
        ## save picture in the output folder
        plt.savefig(f'output/{data_set}.png')


保存訓練好的模型

In [ ]:
import joblib

# 保存模型到文件
joblib.dump(optimized_mlp, f'model/mlp_regressor_model_{data_set}_30_10%.pkl')
print(f"模型已保存到 'model/mlp_regressor_model_{data_set}_30_10%.pkl'")


In [ ]:
import joblib
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
# 載入已保存的模型
loaded_model = joblib.load(f'model/mlp_regressor_model_{data_set}_30_10%.pkl')
print("模型已成功載入")

# 讀取新的測試資料
new_test_data = pd.read_csv(f"data_all/{data_set}.csv")

# 選擇所需的特徵列
selected_columns = ['Albedo_01', 'Albedo_02', 'Albedo_03', 'Albedo_04']
X_new_test = new_test_data[selected_columns]

# 標準化數據
scaler = StandardScaler()
X_new_test = scaler.transform(X_new_test)

# 使用模型進行預測
new_test_predictions = loaded_model.predict(X_new_test)

# 將預測結果添加到資料框
new_test_data['Predicted_Chlorophyll'] = new_test_predictions

# 如果新測試資料包含實際的 Chlorophyll 值，計算評估指標
if 'Chlorophyll' in new_test_data.columns:
    # 計算誤差度量（MAE, RMSE, R2）
    mae = mean_absolute_error(
        new_test_data['Chlorophyll'], new_test_predictions)
    rmse = sqrt(mean_squared_error(
        new_test_data['Chlorophyll'], new_test_predictions))
    r2 = r2_score(new_test_data['Chlorophyll'], new_test_predictions)

    print(f"New Test MAE: {mae}")
    print(f"New Test RMSE: {rmse}")
    print(f"New Test R2: {r2}")

# 保存預測結果到新的CSV文件
new_test_data.to_csv("data_output/new_test_predictions.csv",
                     index=False, encoding='utf-8-sig')
print("新的測試預測結果已保存為 'data/new_test_predictions.csv'")